# FluxBack Demo Notebook

This notebook demonstrates how to use FluxBack for backtesting strategies.


In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'python'))

try:
    import fluxback
    print("FluxBack imported successfully!")
except ImportError as e:
    print(f"Import error: {e}")
    print("\nTo build Python bindings:")
    print("  cd build")
    print("  cmake .. -DBUILD_PYTHON_BINDINGS=ON")
    print("  cmake --build .")
    print("  # Then add build/python to PYTHONPATH or install")


## List Available Indicators


In [ ]:
indicators = fluxback.list_indicators()
for ind in indicators:
    print(f"  - {ind}")


## Run a Backtest


In [ ]:
import os

# Paths relative to notebook directory
strategy_path = os.path.join('..', 'config', 'sma_demo.yaml')
data_path = os.path.join('..', 'demo', 'sample_data.csv')

print(f"Strategy: {strategy_path}")
print(f"Data: {data_path}")

# Run backtest
result = fluxback.run_backtest(strategy_path, data_path)

print("\n=== Backtest Results ===")
print(f"Total Return: {result['total_return_pct']:.2f}%")
print(f"Sharpe Ratio: {result['sharpe_ratio']:.4f}")
print(f"Max Drawdown: {result['max_drawdown_pct']:.2f}%")
print(f"Total Trades: {result['total_trades']}")
print(f"Win Rate: {result['win_rate_pct']:.2f}%")
print(f"Profit Factor: {result['profit_factor']:.4f}")


## Visualize Results


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Create a simple visualization
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Performance metrics
metrics = {
    'Total Return (%)': result['total_return_pct'],
    'Sharpe Ratio': result['sharpe_ratio'],
    'Max Drawdown (%)': result['max_drawdown_pct'],
    'Win Rate (%)': result['win_rate_pct']
}

axes[0].bar(metrics.keys(), metrics.values(), color=['green' if v > 0 else 'red' for v in metrics.values()])
axes[0].set_title('Backtest Performance Metrics')
axes[0].set_ylabel('Value')
axes[0].tick_params(axis='x', rotation=45)

# Trade statistics
trade_stats = {
    'Total Trades': result['total_trades'],
    'Winning Trades': result['winning_trades'],
    'Losing Trades': result['losing_trades']
}

axes[1].bar(trade_stats.keys(), trade_stats.values(), color=['blue', 'green', 'red'])
axes[1].set_title('Trade Statistics')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.show()

print(f"\nInitial Cash: ${result['initial_cash']:.2f}")
print(f"Final Cash: ${result['final_cash']:.2f}")
print(f"Net P&L: ${result['final_cash'] - result['initial_cash']:.2f}")


## Per-Regime Analysis


In [ ]:
if 'trades_by_regime' in result and result['trades_by_regime']:
    print("\n=== Per-Regime Statistics ===")
    for regime, count in result['trades_by_regime'].items():
        pnl = result['pnl_by_regime'].get(regime, 0.0)
        print(f"{regime}: {count} trades, PnL: ${pnl:.2f}")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    regimes = list(result['trades_by_regime'].keys())
    counts = list(result['trades_by_regime'].values())
    pnls = [result['pnl_by_regime'].get(r, 0.0) for r in regimes]
    
    axes[0].bar(regimes, counts, color=['blue', 'orange', 'green'])
    axes[0].set_title('Trades by Regime')
    axes[0].set_ylabel('Number of Trades')
    
    axes[1].bar(regimes, pnls, color=['green' if p > 0 else 'red' for p in pnls])
    axes[1].set_title('PnL by Regime')
    axes[1].set_ylabel('PnL ($)')
    axes[1].axhline(y=0, color='black', linestyle='--', linewidth=0.5)
    
    plt.tight_layout()
    plt.show()
else:
    print("No regime data available.")


## Notes

- This is a demo with sample data. For production use, download real market data from:
  - [Kaggle: Intraday Stock Prices](https://www.kaggle.com/datasets/borismarjanovic/price-volume-data-for-all-us-stocks-etfs)
- The strategy uses SMA crossover (10/20) with adaptive slippage
- Regime detection helps identify market conditions (TREND, VOLATILE, SIDEWAYS)
- Results include comprehensive metrics: Sharpe ratio, max drawdown, win rate, etc.
